# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `pyclanglite` `parser`.

The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g. copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g., `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g., `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for some sequence containers (e.g., `vector` of the `std` namespace) and associative containers (e.g., `set`, `unordered_set` of the `std` namespace).
These template instantiations are done for various *C++* fundamental types (e.g., `int`, `unsigned long int`, `double`) and the `string` of the `std` namespace).
For ordered associative containers only the `std::less` comparator was used.
For the complete procedure refers to the `AutoWIG.py` file situed at the root of the **STL** [repository](https://github.com/StatisKit/STL).



First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

* to detect the version of *Python* installed and save it in the `PYTHON_VERSION` environment variable.

In [2]:
import os
import sys
os.environ['PYTHON_VERSION'] = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

In this notebook, we do not need to import **AutoWIG** since **SCons** is configured to use the **Boost.Python** tool installed with **AutoWIG** that can be used to generate wrappers (see the `../git/STL/src/cpp/SConscript` file).

In [4]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\SConscript
else:
    !pygmentize ../git/STL/src/cpp/SConscript

# -*-python-*-

VECTORS = ['Index',
           'int',
           'double',
           'std::string']

SETS = { 'less': ['Index',
                  'int',
                  'double',
                  'std::string'],
         'none': ['Index']}

HEADER = """\
#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
   

The controller is registered in the `../git/STL/src/cpp/AutoWIG.py` file

In [5]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\AutoWIG.py
else:
    !pygmentize ../git/STL/src/cpp/AutoWIG.py

import autowig
import itertools

def controller(asg, **kwargs):
    autowig.controller.plugin = 'default'
    asg = autowig.controller(asg, **kwargs)
    for function in asg['::statiskit::stl'].functions():
        if function.localname in ['generator', 'insert']:
            parameter = function.parameters[0].qualified_type.desugared_type
            if parameter.is_class:
                function.parent = parameter.unqualified_type
    for cls in asg['class ::std::vector'].specializations(partial = False):
        for method in cls.methods():
            if method.localname in ['resize', 'shrink_to_fit', 'operator[]']:
                if isinstance(method.boost_python_export, bool):
                    method.boost_python_export = False
        for constructor in cls.constructors():
            if not(constructor.nb_parameters == 0 or constructor.nb_parameters == 1 and constructor.parameters[0].qualified_type.unqualified_type == cls):
                if isinstance(constructor.boost_p

Then, in addition to the **STL** library, the **StatisKit.STL** library is needed in order to have access to some functionalities.
To do so, we use available **Conda** recipes.

In [7]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\STL\bin\conda\libstatiskit_stl -c statiskit
else:
    !conda build --python=$PYTHON_VERSION ../git/STL/bin/conda/libstatiskit_stl -c statiskit
#!conda install -y libstatiskit_stl --use-local -c statiskit --force

Attempting to finalize metadata for libstatiskit_stl
INFO:conda_build.metadata:Attempting to finalize metadata for libstatiskit_stl
BUILD START: [u'libstatiskit_stl-1.0.0-h3ddae77_0.tar.bz2']

The following NEW packages will be INSTALLED:

    ca-certificates: 2017.08.26-h1d4fec5_0               
    certifi:         2017.7.27.1-py36h8b7b77e_0          
    libedit:         3.1-heed3624_0                      
    libffi:          3.2.1-h4deb6c0_3                    
    libgcc-ng:       7.2.0-h7cc24e2_2                    
    libstdcxx-ng:    7.2.0-h7a57d05_2                    
    libtoolchain:    1.0.0-py36_0               statiskit
    ncurses:         6.0-h06874d7_1                      
    openssl:         1.0.2l-h9d1a558_3                   
    path.py:         10.3.1-py36he0c6f6d_0               
    pip:             9.0.1-py36h8ec8b28_3                
    python:          3.6.3-hcad60d5_0                    
    python-scons:    3.0.0-py36_0               statiskit
    py

++ ((  0 != 0  ))
++ _CONDA_HOLD=true
++ source /home/main/miniconda/bin/deactivate
+++ [[ -n 4.3.11(1)-release ]]
+++ _SCRIPT_LOCATION=/home/main/miniconda/bin/deactivate
+++ _SHELL=bash
++++ dirname /home/main/miniconda/bin/deactivate
+++ _CONDA_DIR=/home/main/miniconda/bin
+++ case "$(uname -s)" in
++++ uname -s
+++ EXT=
+++ [[ 1 > 0 ]]
+++ key=/home/main/miniconda/conda-bld/libstatiskit_stl_1507481345886/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh
+++ case $key in
+++ shift
+++ [[ 0 > 0 ]]
+++ [[ -n 4.3.11(1)-release ]]
++++ basename /home/main/miniconda/conda-bld/libstatiskit_stl_1507481345886/work/conda_build.sh
+++ [[ conda_build.sh == \d\e\a\c\t\i\v\a\t\e ]]
+++ [[ -z '' ]]
+++ [[ -n 4.3.11(1)-release ]]
++++ basename /home/main/miniconda/conda-bld/libstatiskit_stl_1507481345886/work/conda_build.sh
+++ [[ conda_build.sh == \d\e\a\

scons: Reading SConscript files ...
ModuleNotFoundError: No module named 'autowig':
  File "/home/main/miniconda/conda-bld/libstatiskit_stl_1507481345886/work/SConstruct", line 10:
    SConscript(os.path.join('build', 'src', 'cpp', 'SConscript'), exports="env")
  File "/home/main/miniconda/conda-bld/libstatiskit_stl_1507481345886/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/lib/python3.6/site-packages/SCons/Script/SConscript.py", line 607:
    return method(*args, **kw)
  File "/home/main/miniconda/conda-bld/libstatiskit_stl_1507481345886/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh/lib/python3.6/site-packages/SCons/Script/SConscript.py", line 544:
    return _SConscript(self.fs, *files, **sub

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations (see the `../git/STL/src/cpp/STL.h` file).

In [ ]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\STL.h
else:
    !pygmentize ../git/STL/src/cpp/STL.h

#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
    #endif
#endif

namespace statiskit
{
    typedef unsigned long int Index;
    typedef std::set< Index > Indices;

    namespace stl
    {

        template<class T>
            class Generator
            {
                public:
                    Generat

Once these preliminaries are done, we can proceed to the actual generation of wrappers for the **STL** library.
To do so, we need then to install the *C++* headers. 
This is done using the `cpp` target in **SCons**.

In [ ]:
if is_windows:
    !scons cpp -C ..\git\STL
else:
    !scons cpp -C ../git/STL

scons: Entering directory `/home/main/repository/share/git/STL'
scons: Reading SConscript files ...
Returning non-final recipe for libstatiskit_stl-1.0.0-0; one or more dependencies was unsatisfiable:
Build: libtoolchain
Host: None
Returning non-final recipe for python-statiskit_stl-3.3.1-py27_0; one or more dependencies was unsatisfiable:
Build: libtoolchain, libstatiskit_stl, python-toolchain, libboost
Host: None
Returning non-final recipe for python-statiskit_stl-3.3.1-py27_0; one or more dependencies was unsatisfiable:
Build: libtoolchain, libstatiskit_stl, python-toolchain, libboost
Host: None
Returning non-final recipe for python-statiskit_stl-3.3.1-py27_0; one or more dependencies was unsatisfiable:
Build: libtoolchain, libstatiskit_stl, python-toolchain, libboost
Host: None
Returning non-final recipe for libstatiskit_stl-1.0.0-0; one or more dependencies was unsatisfiable:
Build: libtoolchain
Host: None
Build: libtoolchain
Host: None
Returning non-final recipe for libstatiskit_

Once the headers have been installed in the system, we parse headers with relevant compilation flags.
This is done using the `autowig` target in **SCons**.

In [ ]:
if is_windows:
    !scons autowig -C ..\git\STL
else:
    !scons autowig -C ../git/STL

Here is the list of the generated wrappers (untracked files).

In [ ]:
if is_windows:
    !cd ..\git\STL & git status
else:
    !cd ../git/STL && git status

And here, we present the wrappers for the `std::vector< int >` class.

In [ ]:
if is_windows:
    !pygmentize ..\git\STL\src\py\wrapper\wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp
else:
    !pygmentize ../git/STL/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [ ]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\STL\bin\conda\python-statiskit_stl -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../git/STL/bin/conda/python-statiskit_stl -c statiskit -c conda-forge
!conda install -y python-statiskit_stl --use-local -c statiskit -c conda-forge --force

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
from statiskit.stl import VectorInt
v = VectorInt()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

In [ ]:
list(v)

In [ ]:
v[0]

In [ ]:
v[0] = -2
v[0]

In [ ]:
VectorInt([0, 1])

Here is a report concerning objects wrapped using this notebook.

In [ ]:
import fp17
import os
import pickle
with open(os.path.join(os.environ['SCONSFLAGS'].lstrip('--site-dir='),
                       '..',
                       '..',
                       'autowig',
                       'site',
                       'ASG',
                       'statiskit_stl.pkl'), 'rb') as filehandler:
    asg = pickle.load(filehandler)
fp17.report(asg)